In [170]:
import pandas as pd
import os, random, re, gspread
import numpy as np

In [2]:
sheet_id = '165sp-lWd1L4qWxggw25DJo_njOCvzdUjAd414NSE8co'
sheet_name = 'tz_data'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
print(url)
ds = pd.read_csv(url)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
display(ds)
print('Посмотрим наборы значений кластеров:')
display(ds['cluster'].value_counts().sort_index())

https://docs.google.com/spreadsheets/d/165sp-lWd1L4qWxggw25DJo_njOCvzdUjAd414NSE8co/gviz/tq?tqx=out:csv&sheet=tz_data


,area,cluster,cluster_name,keyword,good (1),count,x,y
0,eligibility,0,Кластер 0,several animated buried,1.0,1260.0,5.772342,12.564796257345005
1,eligibility,0,Кластер 0,singles unusual buyers,1.0,866.0,14.829280,7.8507285727125815
2,eligibility,0,Кластер 0,hawaiian directive,1.0,163.0,11.381856,3.8981370219558604
3,eligibility,0,Кластер 0,dynamics directly,1.0,1146.0,9.980149,6.281427914064545
4,eligibility,1,Кластер 1,decision surgeons montreal,1.0,823.0,3.283940,4.39674063521296
5,eligibility,1,Кластер 1,knives everybody,1.0,1377.0,5.607192,13.155189236409358
6,eligibility,1,Кластер 1,lighting observe,1.0,281.0,11.419919,3.1617111286924686
7,eligibility,1,Кластер 1,payday oriental,1.0,1501.0,8.035720,2.9656951192720844
8,eligibility,1,Кластер 1,vampire injured,0.0,139.0,5.247683,0.7791614555083071
9,eligibility,2,Кластер 2,vampire injured,0.0,139.0,5.247683,0.7791614555083071


Посмотрим наборы значений кластеров:


0    60
1    62
2    61
3    45
Name: cluster, dtype: int64

In [6]:
def rnd_clr(num_gr):
    #создадим словарь значений цветов по палетке Tableau для словосочетаний в кластерах
    
    # Tableau 20 Colors - два варианта кодировок цветов
    
    # tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
    #              (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
    #              (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
    #              (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
    #              (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

    # for i in range(len(tableau20)):  
    #     r, g, b = tableau20[i]  
    #     tableau20[i] = (r / 255., g / 255., b / 255.)
    #     tableau20[i] = ' '.join(str(i) for i in [r / 255., g / 255., b / 255.])
    # Use with plt.plot(…, color=tableau[0],…)
    # From http://tableaufriction.blogspot.ro/2012/11/finally-you-can-use-tableau-data-colors.html
    # With code from http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/

    tableau20 = ['#9edae5', '#17becf', '#dbdb8d', '#bcbd22',  
                 '#c7c7c7', '#7f7f7f', '#f7b6d2', '#e377c2',  
                 '#c49c94', '#8c564b', '#c5b0d5', '#9467bd',  
                 '#ff9896', '#d62728', '#98df8a', '#2ca02c',  
                 '#ffbb78', '#ff7f0e', '#aec7e8', '#1f77b4']
    #можно закомментировать/раскомментировать следующую строку, если нужны разные случайные цвета кластеров в разных area
    #или одинаковые цвета кластеров в разных area
    
    #random.seed('12345')
    return random.sample(tableau20, num_gr)

def clr_col(x):
    tableau20 = rnd_clr(len(x.groupby(x).groups.items()))
    for key, ind_lst in x.groupby(x).groups.items():
        df.loc[ind_lst, 'color'] = tableau20[key]
    return df.loc[x.index, 'color']
def rm_dup(x):
    #display(x)
    x = x.drop_duplicates(subset = 'keyword')
    #display(x)
    return x

In [169]:
#удаляем лишний столбец
df = ds.drop('good (1)', axis=1).copy()

#добавляем столбец color
#df['color'] = np.NAN
#df['color'] = df['color'].astype('object')

#группируем по территории и кластерам и заполняем колонку color
df['color'] = df.groupby(['area'])['cluster'].transform(clr_col)

#группируем по территории и кластерам и убираем дубликаты
print('\nКоличество записей до удаления дубликатов:')
print(len(df))
df = df.groupby(['area']).apply(rm_dup).reset_index(drop=True)
print('Количество записей после удаления дубликатов:')
print(len(df))
df = df.sort_values(['area', 'cluster', 'cluster_name', 'count'], ascending=( True, True, True, False))
df = df.dropna().reset_index(drop=True)
#отфильтруем датафрейм с использованием шаблона regex по символам латиницы нижнего регистра в колонке 'keyword' 
df = df[df['keyword'].str.contains(r'[a-z]+$')==True]
#отфильтруем датафрейм с использованием шаблона regex по символам десятиричной системы счисления в колонке 'y' 
df = df[df['y'].str.contains(r'0[xX][0-9a-fA-F]+')==False].reset_index(drop=True)
display(df)


Количество записей до удаления дубликатов:
228
Количество записей после удаления дубликатов:
221


,area,cluster,cluster_name,keyword,count,x,y,color
0,ar\vr,0.0,Кластер 0,written conflict fabulous,1443.0,2.991167,7.106798807182176,#7f7f7f
1,ar\vr,0.0,Кластер 0,reservations linking,751.0,10.195602,12.259495712252914,#7f7f7f
2,ar\vr,0.0,Кластер 0,interfaces neutral,586.0,10.443533,13.809914970161072,#7f7f7f
3,ar\vr,0.0,Кластер 0,committees parallel,173.0,6.735260,3.613982972088637,#7f7f7f
4,ar\vr,1.0,Кластер 1,postcards looked republic detector,1397.0,10.474474,6.220011967033064,#bcbd22
5,ar\vr,1.0,Кластер 1,michael tobacco,1007.0,6.210880,12.721263957780877,#bcbd22
6,ar\vr,1.0,Кластер 1,filling volunteers academics,773.0,10.833920,8.652736658281233,#bcbd22
7,ar\vr,1.0,Кластер 1,celtic automation,715.0,12.197275,4.723751417014398,#bcbd22
8,ar\vr,2.0,Кластер 2,enquiries implementation,1249.0,3.580718,8.089951461567338,#c7c7c7
9,ar\vr,2.0,Кластер 2,previously standing languages commands,889.0,14.668089,8.467136440992878,#c7c7c7


In [184]:
# gc = gspread.service_account(filename=os.path.join('G:', os.sep,'Мой диск\\IT\\HightSchoolProbation\\crypto-analyzer-378014-709e493cdad7.json'))

# sh = gc.open("Example spreadsheet")

# print(sh.sheet1.get('A1'))

#from oauth2client.service_account import ServiceAccountCredential
from google.oauth2.service_account import Credentials

scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = Credentials.from_service_account_file('jsonFileFromGoogle.json', scopes=scopes)

gc = gspread.authorize(credentials)

spreadsheet_key = 'red_url_code_goes_here'
wks_name = 'Master'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

FileNotFoundError: [Errno 2] No such file or directory: 'jsonFileFromGoogle.json'